In [1]:
import tensorflow as tf
import numpy as np
import psycopg2
import pandas as pd

In [2]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgress")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [3]:
### load dataset
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432")

# usage: specify file location, sample size and seed(used by random) 
filepath = '../Data_git_ignore/clean_csv/article_clean.csv' # 250 rows of FakeNewsCorpus
s = 250                                            # desired sample size
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_sql_query("SELECT * FROM fakenews.article", connection)
#df["content"] = df["content"].astype(str)
# c*reate type_id
#df['type_id'] = df.groupby(['type']).ngroup()

df.head()

,id,domain_id,type_id,url,content_tokenize,content,title,meta_description,scraped_at,updated_at,inserted_at
0,338715,0,0,https://theinternetpost.net/tag/global-war/,"['we', 'are', 'witnessing', 'a', 'great', 'tur...",we are witnessing a great turning inward in th...,THE INTERNET POST,None,2018-01-25,2018-02-02,2018-02-02
1,30948,1,0,http://beforeitsnews.com/spirit/2013/06/more-t...,"['more', 'than', 'a', 'parade', 'and', 'my', '...",more than a parade and my new manifesto of rea...,"More than a parade, and my new manifesto",None,2018-01-25,2018-02-02,2018-02-02
2,819308,1,0,http://beforeitsnews.com/international/2013/01...,"['dallas', 'gun', 'buyback', 'program', 'count...",dallas gun buyback program countered with succ...,Dallas Gun Buyback Program Countered With Succ...,None,2018-01-25,2018-02-02,2018-02-02
3,339605,0,0,https://theinternetpost.net/tag/free-trade-area/,"['by', 'paul', 'j', 'balles', 'many', 'industr...",by paul j balles many industries that started ...,THE INTERNET POST,None,2018-01-25,2018-02-02,2018-02-02
4,24651,1,0,http://beforeitsnews.com/business/2015/11/ente...,"['enterprise', 'vsat', 'market', 'global', 're...",enterprise vsat market global research and ana...,Enterprise VSAT Market Global Research and Ana...,None,2018-01-25,2018-02-02,2018-02-02


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

#df['content'] = df['content'].apply(' '.join)

x = v.fit_transform(df["content"])

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x.toarray(), df["type_id"], test_size=0.2, random_state=0)

MemoryError: 

In [15]:
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train.values, epochs=3)

Train on 9600 samples
Epoch 1/3
9600/9600 [==============================] - 8s 824us/sample - loss: 0.4544 - accuracy: 0.8214
Epoch 2/3
9600/9600 [==============================] - 4s 432us/sample - loss: 0.0955 - accuracy: 0.9795
Epoch 3/3
9600/9600 [==============================] - 4s 425us/sample - loss: 0.0229 - accuracy: 0.9981


In [17]:
# dette for for at trække om modellen er overfittet. val_loss og val_acc skal minde meget om det man får i fittet. 
#Det er ok at loss og acc er lidt højre på testen. De må ikke være fortæt eller for længt væk fra hinaden
val_loss, val_acc = model.evaluate(X_test,y_test.values)
print(val_loss, val_acc)

2400/2400 [==============================] - 5s 2ms/sample - loss: 0.0928 - accuracy: 0.9675
0.09277646052340667 0.9675
